In [1]:
test_try_n=1

In [2]:
import torch
import os
from torch import nn
import json
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import random

os.listdir('./try'+str(test_try_n)+'_result/')

# ['DNS', 'LDAP', 'MSSQL', 'NetBIOS', 'NTP', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDPLag', 'UDP']

['Krum_3-D0']

In [3]:
e_name="Krum_3-D0"   #修改这里AVG-D0

In [4]:
def get_f1_np(y0_score,y1_score):
    p_num=len(y1_score)
    n_num=len(y0_score)
    x_label=np.array([0]*n_num+[1]*p_num)
    x_score=np.array(y0_score+y1_score)
    precision, recall, thresholds= metrics.precision_recall_curve(x_label, x_score, pos_label=1)
    # 参考https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html
    # thresholds是递增序列 precision和recall的长度比thresholds多1 precision的最后一个元素是1 recall的最后一个元素是0
    
    fenzi=2*recall*precision
    fenmu=recall+precision
    f1_scores=[(float(fenzi[i]/fenmu[i]) if fenmu[i]!=0 else -1.0) for i in range(len(fenzi))]
    return precision, recall, thresholds, f1_scores

def calculate_F1(y0_score,y1_score,threshold):
    d={'TP':0,'FP':0,'FN':0,'TN':0}
    for i in y0_score:
        if i<threshold:
            d['TN']+=1
        else:
            d['FP']+=1
    for i in y1_score:
        if i<threshold:
            d['FN']+=1
        else:
            d['TP']+=1
    Accuracy=(d['TP']+d['TN'])/(d['TP']+d['TN']+d['FP']+d['FN'])
    Precision=d['TP']/(d['TP']+d['FP'])
    Recall=d['TP']/(d['TP']+d['FN'])
    F1=2*Precision*Recall/(Precision+Recall)
    return Accuracy,Precision,Recall,F1

In [5]:
load_j = json.load(open("./try"+str(test_try_n)+"_result/"+e_name+"/score.json", 'r'))

for test_type in ['DNS', 'LDAP', 'MSSQL', 'NetBIOS', 'NTP', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDPLag', 'UDP']:
    precision, recall, thresholds, f1_scores=get_f1_np(load_j['BENIGN'],load_j[test_type])
    
    m_index=np.argmax(f1_scores)
    print('Best threshold: ', thresholds[m_index])
    print('Best F1-Score: ', np.max(f1_scores))
    print(test_type,precision[m_index], recall[m_index])
    print('')

y1_score=[]
for test_type in ['DNS', 'LDAP', 'MSSQL', 'NetBIOS', 'NTP', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDPLag', 'UDP']:
    y1_score=y1_score+load_j[test_type]
precision, recall, thresholds, f1_scores=get_f1_np(load_j['BENIGN'],y1_score)

m_index=np.argmax(f1_scores)
print('Best threshold: ', thresholds[m_index])
print('Best F1-Score: ', np.max(f1_scores))
print('[overall]',precision[m_index], recall[m_index])

Best_th=float(thresholds[m_index])

Best threshold:  0.08968929946422577
Best F1-Score:  0.9956082586770693
DNS 0.9971412235563178 0.99408

Best threshold:  0.2893509268760681
Best F1-Score:  0.9993500844890164
LDAP 0.9992202027472857 0.99948

Best threshold:  0.16085591912269592
Best F1-Score:  0.9992252285575755
MSSQL 0.9989306309277526 0.99952

Best threshold:  0.08987925946712494
Best F1-Score:  0.9960805121864534
NetBIOS 0.9972236421405017 0.99494

Best threshold:  0.09140703082084656
Best F1-Score:  0.9955160095992465
NTP 0.9975200554222432 0.99352

Best threshold:  0.08987553417682648
Best F1-Score:  0.9963718979727667
Portmap 0.9972252551863687 0.99552

Best threshold:  0.08988220989704132
Best F1-Score:  0.9981517004355993
SNMP 0.997235087439112 0.99907

Best threshold:  0.11369596421718597
Best F1-Score:  0.998850333403313
SSDP 0.9985608347158648 0.99914

Best threshold:  0.07540317624807358
Best F1-Score:  0.8894576918778836
Syn 0.9505681365802614 0.83573

Best threshold:  0.20259599387645721
Best F1-Score:  0

In [6]:
class TabTransformNet(nn.Module):
    def __init__(self):
        super(TabTransformNet, self).__init__()
        self.net=nn.Sequential(
            # nn.Linear(79, 64,bias=False),
            nn.Linear(66, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 66),
        )

    def forward(self, x):
        out = self.net(x)
        return out

class NeuTraL(nn.Module):
    def __init__(self, num_trans):
        super(NeuTraL, self).__init__()
        self.enc=nn.Sequential(
            nn.Linear(66, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
        )
        self.num_trans=num_trans
        self.trans = nn.ModuleList([TabTransformNet() for _ in range(self.num_trans)])

    def forward(self,x):
        x_T = torch.zeros(x.shape[0],self.num_trans,x.shape[-1]).to(x)
        for i in range(self.num_trans):
            mask = self.trans[i](x)
            mask = torch.sigmoid(mask)
            x_T[:, i] = mask * x
        x_cat = torch.cat([x.unsqueeze(1),x_T],1)  # x_cat形状为[batch_size,self.num_trans+1,79]
        zs = self.enc(x_cat.reshape(-1,x.shape[-1]))
        z_dim=zs.shape[-1]
        return zs.reshape(x.shape[0],self.num_trans+1,z_dim)

In [7]:
def NeuTraL_loss(z,device,y,temp=0.1):
    z=z.to(device)
    z = nn.functional.normalize(z, p=2, dim=-1)
    # 上面这一行 z本来形状为[batch_size,num_trans+1,z_dim] 处理后 每个z[a,b,:]向量都分别被变为长度为1
    
    z_ori = z[:, 0]  # n,z
    z_trans = z[:, 1:]  # n,k-1, z
    batch_size, num_trans, z_dim = z.shape  # 此处的num_trans实际上是num_trans+1

    sim_matrix = torch.exp(torch.matmul(z, z.permute(0, 2, 1) / temp))  # n,k,k
    
    # 下面这一行 减号前的一项形状为[batch_size,num_trans+1,z_dim] 减号后的一项形状为[1,num_trans+1,z_dim]
    # torch.eye()对角矩阵
    mask = (torch.ones_like(sim_matrix).to(device) - torch.eye(num_trans).unsqueeze(0).to(device)).bool()
    # 下面这一行 masked_select返回一维tensor
    sim_matrix = sim_matrix.masked_select(mask).view(batch_size, num_trans, -1)
    trans_matrix = sim_matrix[:, 1:].sum(-1)  # n,k-1

    pos_sim = torch.exp(torch.sum(z_trans * z_ori.unsqueeze(1), -1) / temp) # n,k-1
    K = num_trans - 1
    scale = 1 / abs(K*float(torch.log(torch.tensor(1.0 / K))))

    # loss_tensor = (torch.log(trans_matrix+1e-7) - torch.log(pos_sim+1e-7)) * scale
    p_k=pos_sim/(trans_matrix+1e-7) # n,k-1
    # y原本的形状是[n]
    y=y.view(-1,1)
    p_k_y=(1-y)*p_k+y*(1-p_k)
    l_tensor =-torch.log(p_k_y+1e-7)
    return l_tensor.sum(1)

def e_loss(x,net,device):
    # 测试时 仅使用损失函数的一项 需要传输全为0的y作为参数
    y=torch.zeros(x.shape[0]).float()
    y=y.to(device)
    net = net.to(device)
    x=x.to(device)
    with torch.no_grad():
        z=net(x)
        l=NeuTraL_loss(z,device,y)
    return l

In [8]:
t_num=4
model = NeuTraL(t_num)
model.load_state_dict(torch.load('./try'+str(test_try_n)+'_result/'+e_name+'/model.pt'))
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

C:\Users\riven\AppData\Local\Temp\ipykernel_30964\4051924339.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./try'+str(test_try_n)+'_r

In [9]:
v_x_0=torch.load('./F1/validition/validition_x.pt')
v_x_1=torch.zeros(0,66).float()
for i in ['DNS', 'LDAP', 'MSSQL', 'NetBIOS', 'NTP', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDPLag', 'UDP']:
    v_x_1=torch.cat((v_x_1,torch.load('./F1/validition/validition_'+i+'.pt')),dim=0)
print(v_x_0.shape,v_x_1.shape)

C:\Users\riven\AppData\Local\Temp\ipykernel_30964\3692134789.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  v_x_0=torch.load('./F1/validition/validition_x.pt')
C:\Users

torch.Size([328, 66]) torch.Size([336, 66])


In [10]:
y0_score=e_loss(v_x_0,model,device).cpu().numpy().tolist()
y1_score=e_loss(v_x_1,model,device).cpu().numpy().tolist()
precision, recall, thresholds, f1_scores=get_f1_np(y0_score,y1_score)
th_3=float(thresholds[np.argmax(f1_scores)])
th_3,Best_th

(0.07603519409894943, 0.07553797215223312)

In [11]:
for test_type in ['DNS', 'LDAP', 'MSSQL', 'NetBIOS', 'NTP', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDPLag', 'UDP']:
    print(test_type)
    print('threshold %.5f' % (th_3),end=',')
    print('Accuracy %.5f, Precision %.5f, Recall %.5f, F1 %.5f' % calculate_F1(load_j['BENIGN'],load_j[test_type],th_3))
    
    print('threshold %.5f' % (Best_th),end=',')
    print('Accuracy %.5f, Precision %.5f, Recall %.5f, F1 %.5f' % calculate_F1(load_j['BENIGN'],load_j[test_type],Best_th))

y1_score=[]
for test_type in ['DNS', 'LDAP', 'MSSQL', 'NetBIOS', 'NTP', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDPLag', 'UDP']:
    y1_score=y1_score+load_j[test_type]

print('[overall]')
print('threshold %.5f' % (th_3),end=',')
print('Accuracy %.5f, Precision %.5f, Recall %.5f, F1 %.5f' % calculate_F1(load_j['BENIGN'],y1_score,th_3))

print('threshold %.5f' % (Best_th),end=',')
print('Accuracy %.5f, Precision %.5f, Recall %.5f, F1 %.5f' % calculate_F1(load_j['BENIGN'],y1_score,Best_th))

DNS
threshold 0.07604,Accuracy 0.97727, Precision 0.96956, Recall 0.99948, F1 0.98429
threshold 0.07554,Accuracy 0.97103, Precision 0.96134, Recall 0.99955, F1 0.98007
LDAP
threshold 0.07604,Accuracy 0.97747, Precision 0.96957, Recall 0.99977, F1 0.98444
threshold 0.07554,Accuracy 0.97123, Precision 0.96135, Recall 0.99983, F1 0.98021
MSSQL
threshold 0.07604,Accuracy 0.97747, Precision 0.96957, Recall 0.99977, F1 0.98444
threshold 0.07554,Accuracy 0.97122, Precision 0.96135, Recall 0.99981, F1 0.98020
NetBIOS
threshold 0.07604,Accuracy 0.97745, Precision 0.96957, Recall 0.99973, F1 0.98442
threshold 0.07554,Accuracy 0.97117, Precision 0.96134, Recall 0.99975, F1 0.98017
NTP
threshold 0.07604,Accuracy 0.97618, Precision 0.96951, Recall 0.99796, F1 0.98353
threshold 0.07554,Accuracy 0.96992, Precision 0.96128, Recall 0.99799, F1 0.97929
Portmap
threshold 0.07604,Accuracy 0.97732, Precision 0.96956, Recall 0.99955, F1 0.98433
threshold 0.07554,Accuracy 0.97104, Precision 0.96134, Recall 0

In [12]:
tup=calculate_F1(load_j['BENIGN'],y1_score,th_3)
with open("./try"+str(test_try_n)+"_result/"+e_name+"/th.txt", 'w') as f:
    f.write('threshold\tAccuracy\tPrecision\tRecall\tF1\n')
    f.write(str(th_3))
    f.write('\t')
    f.write(str(tup[0]))
    f.write('\t')
    f.write(str(tup[1]))
    f.write('\t')
    f.write(str(tup[2]))
    f.write('\t')
    f.write(str(tup[3]))

In [13]:
assert False

AssertionError: 